In [1]:
import folium
import pandas as pd
import numpy as np

from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map

In [2]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [3]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [4]:
# Creating folium map object - Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [5]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker) 

In [6]:
# Adding the Launch Sites locations on a map
site_map = folium.Map(location = nasa_coordinate, zoom_start=5)
launch_names = list(launch_sites_df['Launch Site'])
lats = list(launch_sites_df['Lat'])
longs = list(launch_sites_df['Long'])

for site, lat, long in zip(launch_names, lats, longs):
    folium.Circle([lat, long], radius=100, color='blue', fill=True).add_child(folium.Popup(site)).add_to(site_map)
    folium.map.Marker(
    [lat, long],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site,
        )
    ).add_to(site_map)
site_map

### Task 2: Mark the success/failed launches for each site on the map

In [7]:
# Creating markers by Class(0/1). Because the tries happen in same coordinates, should be using Marker clusters. 
marker_cluster = MarkerCluster()

# Creating new column in the df 'marker_color' that corresponds to the Class value (1-green, 0-red)

# First option:
#spacex_df['marker_color'] = np.where(spacex_df['class'] == 1, 'green', 'red')
#spacex_df.head()

# Second option:
# Function to apply conditionally
def color_based_on_class(value):
    if value == 1:
        return 'green'
    else:
        return 'red'

# Apply the function to create the 'color' column
spacex_df['marker_color'] = spacex_df['class'].apply(color_based_on_class)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_44412\3198500781.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spacex_df['marker_color'] = spacex_df['class'].apply(color_based_on_class)


In [8]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.Marker(
    [record.values[1], record.values[2]],
    icon=folium.Icon(color='white', icon_color=record.values[4]))
    marker_cluster.add_child(marker)

site_map

### # TASK 3: Calculate the distances between a launch site to its proximities

In [9]:
# Let's first add a MousePosition on the map to get coordinate for a mouse over a point on the map

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [10]:
 from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [11]:
# We want to estimate the distance of the proximity of launch sites to coastline
coast_lat = 28.56342
coast_lon = -80.56798
distance_coastline = calculate_distance(28.563197, -80.57682, coast_lat, coast_lon)
distance_coastline

0.863953219134235

In [12]:
distance_marker = folium.Marker(
    [coast_lat, coast_lon],
    icon=DivIcon(icon_size=(20,20), icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline))
)
site_map.add_child(distance_marker)
site_map

In [13]:
# Draw a PolyLine between a launch site to the selected coastline point\
coast_coor = [coast_lat, coast_lon]
line = folium.PolyLine(locations=[coast_coor, [28.563197, -80.57682]], weight=1)
site_map.add_child(line)
site_map

In [14]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
coordinates = [[28.56303, -80.57073], [28.10569, -80.63175], [28.42299, -80.76282]]
site_coordinates = [28.563197, -80.57682]

for coordinate in coordinates:
    distance = calculate_distance(site_coordinates[0], site_coordinates[1], coordinate[0], coordinate[1])
    folium.Marker(
    [coordinate[0], coordinate[1]],
    icon=DivIcon(icon_size=(20,20), icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance))).add_to(site_map)
    folium.PolyLine(locations=[coordinate, site_coordinates], weight=1).add_to(site_map)

site_map